# [HydroSAR](https://github.com/HydroSAR/HydroSAR/tree/develop)

Set up the stack with help from HydroSAR [Notebooks](https://github.com/HydroSAR/HydroSAR/tree/develop)

In [1]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

In [2]:
# Download data files from Box
box_data = {'filename': ['S1A_IW_20250205T233956_DVP_RTC10_G_gduned_DD97.zip','S1A_IW_20250217T233955_DVR_RTC10_G_gduned_0491.zip'],
            'link': ['https://ky.box.com/shared/static/2w2x6sj97x2trg0gv9az6k16o0v9zgbl.zip', 'https://ky.box.com/shared/static/w3zruxeczm2zpl9mepixbpp6nxszxzg1.zip']
            }

df = pd.DataFrame(data=box_data)
df






,filename,link
0,S1A_IW_20250205T233956_DVP_RTC10_G_gduned_DD97...,https://ky.box.com/shared/static/2w2x6sj97x2tr...
1,S1A_IW_20250217T233955_DVR_RTC10_G_gduned_0491...,https://ky.box.com/shared/static/w3zruxeczm2zp...


In [ ]:
def data_download():

### [Subset Data Stack](https://github.com/HydroSAR/HydroSAR/blob/develop/notebooks/2-Subset_Data_Stack.ipynb)